In [1]:
import os
import numpy as np
import torch
import torch.optim as optim
import torch.distributions as D
from tqdm import tqdm, trange
from src.svgd import SVGD
from src.gsvgd import FullGSVGDBatch
from src.kernel import RBF, BatchRBF
from src.utils import plot_particles
from src.Tmy_svgd import etmySVGD
from src.manifold import Grassmann
from src.s_svgd import SlicedSVGD
from src.MP_SVGD import mp_shape_SVGD
import matplotlib.pyplot as plt
from src.rand_mysvgd import mySVGD
from scipy.stats import energy_distance
import pickle
import argparse
import time
import torch.autograd as autograd

/home/zhoujk/anaconda3/envs/AG/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
parser = argparse.ArgumentParser(description='Running xshaped experiment.')
parser.add_argument('--dim', type=int,default=10000, help='dimension')
parser.add_argument('--effdim', type=int, default=-1, help='dimension')
parser.add_argument('--lr', type=float,default=0.01, help='learning rate')
parser.add_argument('--lr_g', type=float, default=0.1, help='learning rate for g')
parser.add_argument('--delta', type=float,default=0.01, help='stepsize for projections')
parser.add_argument('--T', type=float, default=1e-4, help='noise multiplier for projections')
parser.add_argument('--nparticles', type=int,default=100, help='no. of particles')
parser.add_argument('--epochs', type=int, default=20,help='no. of epochs')
parser.add_argument('--nmix', type=int, default=2, help='no. of modes')
parser.add_argument('--metric', type=str, default="energy", help='distance metric')
parser.add_argument('--noise', type=str, default="True", help='whether to add noise')
parser.add_argument('--kernel', type=str, default="rbf", help='kernel')
parser.add_argument('--gpu', type=int, default=7, help='gpu')
parser.add_argument('--seed', type=int, default=0, help='random seed') 
parser.add_argument('--suffix', type=str, default="", help='suffix for res folder')
parser.add_argument('--m', type=int, help='no. of projections')
parser.add_argument('--save_every', type=int, default=200, help='step intervals to save particles')
parser.add_argument('--method', type=str, default="all", help='which method to use')


args = parser.parse_args([])
device = torch.device(f'cuda:{args.gpu}' if args.gpu != -1 else 'cpu')
dim = args.dim
lr = args.lr
delta = args.delta
T = args.T
nparticles = args.nparticles
epochs = args.epochs
seed = args.seed
eff_dims = 3
nmix = args.nmix
add_noise = True if args.noise == "True" else False
radius = 5
save_every = args.save_every
print(f"Running for dim: {dim}, lr: {lr}, nparticles: {nparticles}")

Running for dim: 10000, lr: 0.01, nparticles: 100


In [3]:
def mix_gauss_experiment(mixture_dist, means, correlation):
    '''Mixture of Multivariate gaussian with cov matrices being the identity.
    Args:
        mixture_dist: torch.distributions.Categorical-like instance for the 
            probability of each component in the mixture.
        means: Tensor of shape (nmix, d), where nmix is the number of components 
            and d is the dimension of each component.
        correlation: Float between 0 and 1 for the magnitude of correlation between
            the first two dims.
    '''
    nmix, dim = means.shape
    nmix = nmix + 1
    
    # create multibatch multivariate Gaussian
    cov1 = torch.eye(dim, device=device)
    cov1[:2, :2] = torch.Tensor([[2, 0.01],[0.01, 2]])
   
    
    #cov1[dim-2:, dim-2:] = torch.Tensor([[1, correlation], [correlation, 1]])
    cov2 = torch.eye(dim, device=device)
    cov2[:2, :2] = torch.Tensor([[2, 0], [0, 2]])
    #cov2[dim-2:, dim-2:] = torch.Tensor([[1, 0], [0, 1]])
   
    mix_cov = torch.stack((cov1, cov2))
   
    comp = D.MultivariateNormal(means.to(device), mix_cov)

    distribution = D.mixture_same_family.MixtureSameFamily(mixture_dist, comp)   
    return(distribution)

In [4]:
mix_means_x = torch.zeros((2, 100), device=device)
mix_means_x[:, :2] = 0

distribution = mix_gauss_experiment(
    mixture_dist=D.Categorical(torch.ones(mix_means_x.shape[0], device=device)),
    means=mix_means_x,
    correlation=0.01
)

In [5]:
x_target = distribution.sample((nparticles, ))
        # sample from variational density
torch.manual_seed(235)
x_init = 2 + np.sqrt(2) * torch.randn(nparticles, *distribution.event_shape, device=device)
Kernel = RBF
kernel = Kernel(method="med_heuristic")

In [6]:
def score(X):
    X_cp = X.detach().requires_grad_(True)
    log_prob = distribution.log_prob(X_cp)
    score_func = autograd.grad(log_prob.sum(), X_cp, retain_graph=False)[0]
            
    score_func1 = score_func.clone().detach()
    return score_func1

def svgd_kernel(theta_ab):
        """gaussian rbf kernel function"""

        X_cp = theta_ab.clone().requires_grad_()
       
        Y = theta_ab.clone()
        with torch.no_grad():
            kernel.bandwidth(theta_ab, theta_ab)
        K_XX = kernel(X_cp, Y)
        
        grad_K = -autograd.grad(K_XX.sum(), X_cp, retain_graph=False)[0] 
        
        
        

        return K_XX, grad_K

In [7]:

    
def del_tensor_ele( arr, index):
    index = index.to(torch.int)

    # print(index)
    arr1 = arr[0:index]
    arr2 = arr[index+1:]
    return torch.cat((arr1, arr2), dim=0)

def divide( theta,  k):
    theta1 = theta.clone()
    theta1 = theta1 - torch.mean(theta1, 0)
    list = torch.norm(theta1, p=2, dim=0)
    max_list = torch.topk(list, k=k, dim = 0, largest=True, sorted=False)
    support = max_list.indices
    print(support)
    min_list = torch.topk(list, k=theta.shape[1]-k, dim = 0, largest=False, sorted=False)
    blanket = min_list.indices
    

    return blanket, support




def svgd_kernel( theta_ab):
    """gaussian rbf kernel function"""

    X_cp = theta_ab.clone().requires_grad_()
    
    Y = theta_ab.clone()
    
    kernel.bandwidth(theta_ab, theta_ab)
    K_XX = kernel(X_cp, Y)
    
    grad_K = -autograd.grad(K_XX.sum(), X_cp)[0] 
    
    
    

    return K_XX, grad_K







def power(theta, part, s, lnpgrad):
    theta_ab1 = torch.cat([theta[:,part],theta[:,s].view(theta.shape[0], 1)], 1)
    
            
            
    lnpgrad_j1 = lnpgrad[:,s]
            # calculating the kernel matrix
    kxy1, repulsion1 = svgd_kernel(theta_ab1) 
    
    
    attraction1 = torch.matmul(kxy1, lnpgrad_j1) 
    return  attraction1 + repulsion1[:,-1]

def update_blanket( theta, lnpgrad, k, lr ):
    
            
    blanket , support = divide(theta, k)
    
                    
            
    theta_ab = torch.cat([theta[:,blanket],theta[:,support]], 1)
            
    lnpgrad_blanket = lnpgrad[:,blanket]
                # calculating the kernel matrix
    kxy, repulsion = svgd_kernel(theta_ab) 
        
    
    attraction = torch.matmul(kxy, lnpgrad_blanket) 
                

    theta[:,blanket] = theta[:,blanket]  + 0.01*lr*(attraction + repulsion[:, 0:blanket.size(0)]) / theta_ab.size(0)

    return theta, blanket, support


def update_dim(score, theta, k, lr):

    lnpgrad = score(theta)
            
    theta, blanket, support = update_blanket(theta,  lnpgrad, k, lr )

        
    grad1_theta = theta.clone().detach()
    grad2_theta = theta.clone().detach()


    iterator_dim = range(theta.shape[1])
    s = -1

    for j in iterator_dim:
        s = s + 1
        
        if s in blanket:
            a = torch.where(blanket == s)[0]

            blanket = del_tensor_ele(blanket, a)
        if s in support:
            a = torch.where(support == s)[0]

            support = del_tensor_ele(support, a)

    #### STEP2 blanket_to_d
        
        grad1 = power(grad1_theta, blanket, s, lnpgrad)

    ### STEP3 SUPPORT_to_d
        grad2 = power(theta, support, s, lnpgrad)
        


    
    
        vec1 = 0.02*lr*grad1 / (blanket.shape[0] + 1)
        vec2 = lr*grad2 / (support.shape[0] + 1)
    
        grad1_theta[:,s] =  vec1 
        
    
        grad2_theta[:,s] = vec2 
        
    
    


    return grad1_theta, grad2_theta



    
    






def update( x0,score, n_iter = 1000, k = 8, lr=0.001):

    
    theta = x0.clone().detach()
    


    iterator = tqdm(range(n_iter)) 
        
    
    vector1 = torch.zeros_like(theta).detach()

    for  i in iterator:
        a, b = update_dim(score, theta, k, lr)
        opt =  optim.Adam([x0], lr=lr)
        opt.zero_grad()
        x0.grad = a
        opt.step()
        opt.zero_grad()
        x0.grad = b
        opt.step()
    return theta

In [8]:
x0 = x_init.clone().detach().to(device)
update( x0,score, n_iter = 10000, k = 2, lr=0.001)
    

  0%|          | 0/10000 [00:00<?, ?it/s]

tensor([75, 67], device='cuda:7')


  0%|          | 1/10000 [00:00<1:28:08,  1.89it/s]

tensor([75, 67], device='cuda:7')


  0%|          | 2/10000 [00:00<1:18:47,  2.12it/s]

...

  3%|▎         | 291/10000 [01:42<55:42,  2.90it/s]

tensor([75, 67], device='cuda:7')


  3%|▎         | 292/10000 [01:43<55:50,  2.90it/s]

tensor([75, 67], device='cuda:7')


  3%|▎         | 293/10000 [01:43<55:53,  2.89it/s]